# Submit Jobs
## AML Cluster

In [ ]:
# Connect to workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="...",
    resource_group_name="...",
    workspace_name="...",
)

### Compute Resource
We search for an existing cluster. If it does not exist, we create a new one.

Check your Compute page in Azure ML to confirm the process is successful.

In [ ]:
# Create a compute resource

from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "cpu-cluster"

try:
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is.")

except Exception:
    print("Creating a new cpu compute target...")

    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="STANDARD_DS3_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )
    print(f"{cpu_cluster.name} will be created... compute size {cpu_cluster.size}...", end='')
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)
    print("Done!")

### Create a Job Environment
We create an environment that has the packages we need installed. We register it for future use.

Steps:

1. Create an empty folder for the config file.
2. Create a `conda.yml` file containing the requirements of your environment.
3. Create an environment according to the `conda.yml` file and register it to the workspace.

Check your Environments page in Azure ML to confirm the process is successful.

In [ ]:
import os

dependencies_dir = "./03-job-cluster/dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [ ]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - xlrd==2.0.1
    - mlflow== 1.26.1
    - azureml-mlflow==1.42.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0
    - matplotlib

In [ ]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-scikit-learn"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Bike Share pipeline",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment {pipeline_job_env.name} registered to workspace. "
    f"Environment version is {pipeline_job_env.version}."
)

### Command Job
We create an Azure ML command job to train a model for credit default prediction. The command job is used to run a training script in the created environment on the created compute resource. 

The training script handles the data preparation, training and registering of the trained model. In this tutorial, you'll create a Python training script.

Command jobs can be run from CLI, Python SDK, or studio interface. In this tutorial, you'll use the Azure Machine Learning Python SDK v2 to create and run the command job.

Steps:

1. Create an empty folder to contain the code. (This is a good practice, both from organization aspect and performance.)
2. Write your scripts in this folder.

In [ ]:
import os

train_src_dir = "./03-job-cluster/src"
os.makedirs(train_src_dir, exist_ok=True)

In [ ]:
%%writefile {train_src_dir}/parser.py
import os
import argparse


def parse_args():
    # input and output arguments
    parser = argparse.ArgumentParser()
    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float)
    parser.add_argument("--registered_model_name", type=str, help="model name")
    args = parser.parse_args()
    return args

In [ ]:
%%writefile {train_src_dir}/main.py
import os
import argparse
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from parser import parse_args


def data_prep(data, test_train_ratio):
    print("input data:", data)
    
    df = pd.read_csv(data, index_col=0)
    df['hr_sin'] =  df['hr'].apply(lambda x: np.sin(2 * np.pi * x / 24))
    df['hr_cos'] =  df['hr'].apply(lambda x: np.cos(2 * np.pi * x / 24))
    df['temp2'] = (df['temp'] - 0.6)**2
    X = df[['hr', 'hr_sin', 'hr_cos', 'temp', 'temp2', 'hum', 'windspeed','workingday', 'weathersit' ]]
    y = df['cnt']
    mlflow.log_metric("num_samples", X.shape[0])
    mlflow.log_metric("num_features", X.shape[1] )

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_train_ratio,)
    return X_train, X_test, y_train, y_test


def train_model(X_train, X_test, y_train, y_test, n_estimators, learning_rate):

    # convert the dataframe values to array
    X_train = X_train.values
    X_test = X_test.values

    print(f"Training with data of shape {X_train.shape}")

    # clf = GradientBoostingRegressor( n_estimators=n_estimators, learning_rate=learning_rate)
    clf = RandomForestRegressor( n_estimators=n_estimators, )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print(r2_score(y_test, y_pred))
    return clf


def main():
    """Main function of the script."""

    args = parse_args()
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    # Start Logging
    mlflow.start_run()

    # enable autologging
    mlflow.sklearn.autolog()

    X_train, X_test, y_train, y_test = data_prep(args.data, args.test_train_ratio)

    model = train_model(X_train, X_test, y_train, y_test, args.n_estimators, args.learning_rate)

    # Registering the model to the workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=model,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )

    # Saving the model to a file
    mlflow.sklearn.save_model(
        sk_model=model,
        path=os.path.join(args.registered_model_name, "trained_model"),
    )
    
    # Stop Logging
    mlflow.end_run()

if __name__ == "__main__":
    main()

### Configure and Submit
Create a command job from the files and submit it for execution.

After the job is submitted, you can check the status by going to Jobs page of Azure ML and finding the experiment named `experiment_name`. There, click on the most recent run.

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "job_model"
experiment_name="bike-share-exp"
data = Input(type="uri_file",
    path="azureml:hourly:1",
)

job = command(
    inputs={
        "data": data,
        "test_train_ratio": 0.2,
        "learning_rate": 0.25,
        "registered_model_name": registered_model_name,
    },
    code= train_src_dir,  # location of source code
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    environment=f"{custom_env_name}@latest",
    compute=cpu_compute_target,
    experiment_name=experiment_name,
    display_name="cluster-job",
)

# Submit the job
ml_client.create_or_update(job)

### Recommendations

As you become more familiar with Azure ML and your code base, it is a good practice to get away from Notebook and start writing python code. This is the best practice no matter you use Azure ML or not. 

We recommend using an IDE such as Visual Studio Code for such task. It provides a better experience for developing code. 

Real projects can have multiple files with dependencies. And handling them in notebook is not recommended.

Using Azure ML SDK and CLI allows you to directly deploy code from your local machine (no need to use compute instance), if that is desired. 

### Next Steps
If you are interested in deploying this model as an endpoint, [click here](https://learn.microsoft.com/en-us/azure/machine-learning/tutorial-azure-ml-in-a-day).